In [1]:
import random
import numpy as np
from IPython.display import clear_output
import sys
import copy

from tqdm import tqdm

class Alak():

  @staticmethod
  def check_states(our, opponent, board_state):
    try:
      assert our in ['x','o'] and opponent in ['x','o'] and our!=opponent, "Sides not right"
      board_state_characters_ = set(board_state)
      assert 'o' in board_state_characters_ and  'x' in board_state_characters_ and  '_' in board_state_characters_ and  len(board_state_characters_) == 3, "Broad State is not right"
    except AssertionError as error:
      print(error)
      assert False, "Not right intiation"
    
  def __init__(self, our, board_state = ['xxxxx____ooooo'], to_print=True, model=None):
      opponent = 'o' if our=='x' else 'x'
      Alak.check_states(our, opponent, board_state)
      self.our = our
      self.opponent = opponent
      self.board_state = list(board_state)
      self.round = -1
      self.all_rounds = []
      self.result = None
      self.to_print = to_print
      self.model = model

  def parse_move(self, player, opponent, move):
    initial_postion_str = move[0]
    final_postion_str = move[-1]
    
    try:
      initial_postion = int(initial_postion_str, 16)
      final_postion = int(final_postion_str, 16)
      assert initial_postion >=0 and initial_postion <= 13, "Initial Position Incorrect - " + str(initial_postion)
      assert final_postion >=0 and final_postion <= 13, "Final Position Incorrect - " + str(initial_postion)
      assert initial_postion != final_postion, "Don't be oversmart. Move to a new position"
      assert self.board_state[initial_postion] == player, "Hey, you can only move your coins"
      assert self.board_state[final_postion] == "_", "You can't just land over another coin"
      
      return initial_postion, final_postion
    
    except ValueError as e:
      print("ValueError in your input", e)
      return False
    except AssertionError as e:
      print(e)
      return False
    except Exception as e:
      print(e)  
      return False
        

  def get_kill_counts(self, killer, victim, final_postion, verbose=False, board=None, return_board=False):
    if board == None:
      board = self.board_state
    kill_count = 0
    sides = [-1, 1]
    for side in sides:
      may_kill_list = []
      if not ((final_postion == 13 and side == 1) or (final_postion == 0 and side == -1)):
        final_postion_s = final_postion + side
        
        assert final_postion_s>=0 and final_postion_s<=13, "You are going out of the way "+str(final_postion_s)
        
        while final_postion_s > 0 and final_postion_s < 13 and board[final_postion_s] == victim:
          may_kill_list.append(final_postion_s)
          final_postion_s +=  side
        if board[final_postion_s] == killer:

          if len(may_kill_list) > 0 and verbose:
            if self.to_print:
              self.print_state(board)

          for index_ in may_kill_list:
            kill_count += 1
            board[index_] = "_"
    if return_board:
      return kill_count, board
    else:
      return kill_count

  def get_sucide_counts(self, killer, victim, initial_postion, final_postion, sucide_allowed, verbose=False, board=None, return_board=False):
    if board == None:
      board = self.board_state
    is_move_valid = True
    sucide_count = 0
    may_die_list = [final_postion]

    if not (final_postion == 13 or final_postion == 0):
      final_postion_s = final_postion - 1
      while final_postion_s > 0 and final_postion_s < 13 and board[final_postion_s] == victim:
        may_die_list.append(final_postion_s)
        final_postion_s -= 1
      if board[final_postion_s] == killer:
        final_postion_s = final_postion + 1
        while final_postion_s > 0 and final_postion_s < 13 and board[final_postion_s] == victim:
          may_die_list.append(final_postion_s)
          final_postion_s +=  1

        if board[final_postion_s] == killer:
          if len(may_die_list) > 0:
            if sucide_allowed:
              if verbose:
                if self.to_print:
                  self.print_state(board)
              for index_ in may_die_list:
                sucide_count += 1
                board[index_] = "_"
            else:
              is_move_valid = False
              board[initial_postion] = victim
              board[final_postion] = "_"
              if self.to_print:
                print("Sucide moves not allowed!. Try again")
              self.print_state(board)
    if return_board:
      return is_move_valid, -sucide_count, board
    else:
      return is_move_valid, -sucide_count

  def go_kill_or_die(self, initial_postion, final_postion, player, opponent, verbose=False, check_kills=True, check_sucide=True, sucide_allowed=True, board=None, return_board=False):
    gains = 0
    is_move_valid = True
    if check_kills:
      if return_board:
        gains, res_board = self.get_kill_counts(player, opponent, final_postion, verbose, board, return_board)
      else:
        gains = self.get_kill_counts(player, opponent, final_postion, verbose, board, return_board)
    
    if gains == 0 and check_sucide:
      if return_board:
        is_move_valid, gains, res_board = self.get_sucide_counts(opponent, player, initial_postion, final_postion, sucide_allowed, verbose, board, return_board)
      else:
        is_move_valid, gains = self.get_sucide_counts(opponent, player, initial_postion, final_postion, sucide_allowed, verbose, board, return_board)

    if return_board:
      return is_move_valid, gains, res_board
    else:
      return is_move_valid, gains

  def print_state(self, board=None):
    if self.to_print:
      if board == None:
        board = self.board_state
      print("Board:\t", ''.join(board))
      print(" " * 5,"\t","0123456789abcd\n")

  def more_to_play(self, curr_round):
    our_count = self.board_state.count(self.our)
    opponent_count = self.board_state.count(self.opponent)
    o_count = self.board_state.count('o')

    assert not (our_count <= 1 and opponent_count <= 1), "Somehow, everyone lost. LOL" #should never happen
    
    if our_count<=1:
      self.result = -1
      if self.to_print:
        print("You have lost!")
      if len(curr_round) == 1:
        curr_round.append(''.join(self.board_state))
        self.all_rounds.append(np.asarray(curr_round))  
      return False
    elif opponent_count<=1:
      self.result = 1
      if len(curr_round) == 1:
        curr_round.append(''.join(self.board_state))
        self.all_rounds.append(np.asarray(curr_round))  
      if self.to_print:
        print("You have won!")
      return False

    return True 

  def random_play(self, player):
    possible_ip = [ip for ip in range(len(self.board_state)) if self.board_state[ip] == player]
    possible_fp = [fp for fp in range(len(self.board_state)) if self.board_state[fp] == '_']
    initial_postion = random.choice(possible_ip)
    final_postion = random.choice(possible_fp)
    return initial_postion, final_postion

  def human_move(self, prompt, player, opponent):
    move = input(prompt)
    parse_results = self.parse_move(player, opponent, move)
    while not parse_results:
      move = input(prompt)
      parse_results = self.parse_move(player, opponent, move)
    
    return parse_results

  def ai_find_best_move(self, player, opponent, as_str=True, sucide_allowed=False):
    assert self.model, "No AI model assigned"
    possible_ips = [ip for ip in range(len(self.board_state)) if self.board_state[ip] == player]
    possible_fps = [fp for fp in range(len(self.board_state)) if self.board_state[fp] == '_']
    best_ip = None
    best_fp = None
    best_move_score = -float('inf')
    for possible_ip in possible_ips:
      for possible_fp in possible_fps:
        temp_board = copy.deepcopy(self.board_state)
        temp_board[possible_ip] = "_"
        temp_board[possible_fp] = player
        move_valid, gains, res_board = self.go_kill_or_die(possible_ip, possible_fp, player, opponent, verbose=False, board=temp_board, sucide_allowed=sucide_allowed, return_board=True)
        if move_valid:
          score = self.model.predict(''.join(temp_board), player)
          if score > best_move_score:
            best_move_score = score
            best_ip = possible_ip
            best_fp = possible_fp
    
    assert best_ip != None, "Your model did not give a ip"
    assert best_fp != None, "Your model did not give a fp"

    best_ip_str = str(hex(best_ip)[-1])
    best_fp_str = str(hex(best_fp)[-1])
    
    return best_ip_str + best_fp_str
    

  def ai_move(self, player, opponent):
    move = self.ai_find_best_move(player, opponent)
    parse_results = self.parse_move(player, opponent, move)
    while not parse_results:
      move = self.ai_find_best_move(player, opponent)
      parse_results = self.parse_move(player, opponent, move)
    return parse_results

  def play(self, verbose=False, sucide_allowed=False, x_mode='random', o_mode='random'):

    if self.to_print:
      print("Starting Game:")
      print("Your side is",self.our,"\n")
    game_left = True

    while(game_left):
      self.round += 1
      if self.to_print:
        print("-"*23)
        # clear_output(wait=True)
        print("Round:\t", self.round)
        self.print_state()
      
      players = ['x','o']
      curr_round = []
      for player in players:
        if game_left and self.more_to_play(curr_round): 
          played = False
          while(not played):
            opponent = 'o' if player == 'x' else 'x'
            prompt = player+":"

            if player == 'x':
              if x_mode == 'ai':
                initial_postion, final_postion = self.ai_move(player, opponent)
              elif x_mode == 'human':
                initial_postion, final_postion = self.human_move(prompt, player, opponent)
              elif x_mode == 'random':
                initial_postion, final_postion = self.random_play(player)
            
            elif player == 'o':
              if o_mode == 'ai':
                initial_postion, final_postion = self.ai_move(player, opponent)
              elif o_mode == 'human':
                initial_postion, final_postion = self.human_move(prompt, player, opponent)
              elif o_mode == 'random':
                initial_postion, final_postion = self.random_play(player)
            
            if self.to_print:
              print(player,":",hex(initial_postion)[2:],"==>",hex(final_postion)[2:])
            
            self.board_state[initial_postion] = "_"
            self.board_state[final_postion] = player

            move_valid, gains = self.go_kill_or_die(initial_postion, final_postion, player, opponent, verbose, sucide_allowed=sucide_allowed)

            if move_valid:
              played = True
              curr_round.append(''.join(self.board_state))
              if len(curr_round) == 2:
                self.all_rounds.append(np.asarray(curr_round))
              self.print_state()
              if self.to_print:
                print("gain:", gains)   

        else:
          game_left = False
        
  def get_game_data(self):

    if self.our == 'x':
      if self.result == 1: # x has won
        fill_val = 1
      else:
        fill_val = -1

      x_labels = np.full(self.round + 1, fill_val)
      o_labels = np.full(self.round + 1, -fill_val)

    else:
      if self.result == 1: # o has won
        fill_val = 1
      else:
        fill_val = -1

      o_labels = np.full(self.round + 1, fill_val)
      x_labels = np.full(self.round + 1, -fill_val)
    
    all_rounds_np = np.asarray(self.all_rounds)

    return all_rounds_np, x_labels, o_labels

  
  @staticmethod
  def test_capture(off_side = ['x', 'x', 'x', 'x', 'o', 'o', 'o', 'o'],
                  board_list = ['xoxoxx________', 'xooxooxx______', '__xoo__oxo____', '__xoooox_o____', '_o__ooxxxo__o_', '___xoxxxoo____', '___x_oxxoo_____', '____xoxx___x___'],
                  board_expect = ['x_x_xx________', 'x__x__xx______', '__xoo__o_o____', '__x____x_o____', '_o__oo___o__o_', '___xo___oo____', '___x_o__oo_____', '____x_xx___x___'], 
                  verbose = False,
                  model=None):
    
    # play some random game
    alak = Alak('x', 'xxxxx____ooooo', to_print=False, model=model)
    alak.play(sucide_allowed=True, verbose=False, x_mode='random', o_mode='random')

    if model:
      alak = Alak('x', 'xxxxx____ooooo', to_print=False, model=model)
      alak.play(sucide_allowed=True, verbose=False, x_mode='ai', o_mode='ai')
      alak = Alak('x', 'xxxxx____ooooo', to_print=False, model=model)
      alak.play(sucide_allowed=True, verbose=False, x_mode='random', o_mode='ai')
      alak = Alak('x', 'xxxxx____ooooo', to_print=False, model=model)
      alak.play(sucide_allowed=True, verbose=False, x_mode='ai', o_mode='random')
    

    assert len(off_side) == len(board_list) and len(board_list) == len(board_expect)
    
    test_pass_count = 0
    
    for i in range(len(off_side)):
      player = off_side[i]
      opponent = 'o' if player == 'x' else 'x'
      alak = Alak(player, board_list[i]) #it should not matter who is our
      # as we don't have the final position but the kill and sucide check needs that, lets try for all the possible final position of player
      gains = 0
      for j in range(len(alak.board_state)):
        if alak.board_state[j] == player:
          gains += alak.go_kill_or_die(None, j, player, opponent, check_kills = True, check_sucide=False, sucide_allowed=True)[1]
     
      # no kills, check sucide
      if gains == 0:
        for j in range(len(alak.board_state)):
          if alak.board_state[j] == player:
            gains += alak.go_kill_or_die(None, j, player, opponent, check_kills = False, check_sucide=True, sucide_allowed=True)[1]

      #all check, board should look as expected
      board_state_str = ''.join(alak.board_state)
      board_expect_str = board_expect[i]

      if verbose:
        print("RES:", board_state_str)
        print("EXP:", board_expect_str)

      if board_state_str == board_expect_str:
        if verbose:
          print("Pass")
        test_pass_count+=1
      else:
        if verbose:
          print("Fail")

      if verbose:
        print("-"*23)

    print(test_pass_count, "out of ", len(off_side), "tests passed.")

Alak.test_capture()


8 out of  8 tests passed.


In [2]:
class BaseModel():
  def __init__(self, x_move_scores={}, o_move_scores={}):
    self.x_move_scores = x_move_scores
    self.o_move_scores = o_move_scores
  
  def train(self, n_samples=1, thres=float('inf'), x_mode='random', o_mode='random', ai_model=None):
    if x_mode == 'ai' or o_mode == 'ai':
      assert ai_model is not None, "For ai mode, give an ai model" 
    our_side = 'o'
    start_board = 'xxxxx____ooooo'
    alak = Alak(our_side, start_board, to_print=False, model=ai_model)
    alak.play(sucide_allowed=False, verbose=False, x_mode=x_mode, o_mode=o_mode)
    X, yx, yo = alak.get_game_data()
    if X.shape[0] == yx.shape[0]-1:
      yx = yx[:-1]
      yo = yo[:-1]
    assert X.shape[0] == yo.shape[0], "X and y not equal"
    for _ in tqdm(range(n_samples)):
      alak = Alak(our_side, start_board, to_print=False, model=ai_model)
      alak.play(sucide_allowed=False, verbose=False, x_mode=x_mode, o_mode=o_mode)
      if alak.round<thres:
        X_curr, yx_curr, yo_curr = alak.get_game_data()
        if X_curr.shape[0] == yx_curr.shape[0]-1:
          yx_curr = yx_curr[:-1]
          yo_curr = yo_curr[:-1]
        assert X_curr.shape[0] == yo_curr.shape[0], "X and y not equal"
        X = np.append(X, X_curr, axis=0)
        yx = np.append(yx, yx_curr, axis=0)
        yo = np.append(yo, yo_curr, axis=0)

    X_x = X[:,0]
    X_o = X[:,1]

    print("Total moves in training data", X.shape[0])
    print("Total unique x moves in training data", len(set(X_x)))
    print("Total unique o moves in training data", len(set(X_o)))

    for X_x_value in tqdm(set(X_x)):
      self.x_move_scores[X_x_value] = yx[X_x==X_x_value].sum()

    for X_o_value in tqdm(set(X_o)):
      self.o_move_scores[X_o_value] = yo[X_o==X_o_value].sum()

  def predict(self, board, player):
    if player == 'x':
      return self.x_move_scores.get(board,-0.1)
    else:
      return self.o_move_scores.get(board,-0.1)

In [ ]:
base_model = BaseModel()
base_model.train(n_samples=1000000, thres=15)

 18%|█▊        | 178158/1000000 [34:13<11:54:00, 19.18it/s]

In [ ]:
win_count = 0
for j in tqdm(range(10000)):
  alak = Alak('x', 'xxxxx____ooooo', to_print=False, model=base_model)
  alak.play(sucide_allowed=True, verbose=False, x_mode='ai', o_mode='random')
  if alak.result == 1:
    win_count += 1

print("Win Percentage",(win_count/j) * 100)

100%|██████████| 10000/10000 [00:38<00:00, 260.79it/s]

Win Percentage 81.2081208120812


In [ ]:
print("Win Percentage",(win_count/j) * 100)

Win Percentage 81.2081208120812


In [ ]:
import pickle

with open('xmodel.pickle', 'wb') as handle:
    pickle.dump(base_model.x_move_scores, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('omodel.pickle', 'wb') as handle:
    pickle.dump(base_model.o_move_scores, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
alak = Alak('x', 'xxxxx____ooooo', to_print=True, model=base_model)
alak.play(sucide_allowed=True, verbose=False, x_mode='ai', o_mode='human')

Starting Game:
Your side is x 

-----------------------
Round:	 0
Board:	 xxxxx____ooooo
      	 0123456789abcd

x : 3 ==> 7
Board:	 xxx_x__x_ooooo
      	 0123456789abcd

gain: 0


KeyboardInterrupt: ignored